In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import mysql.connector
%matplotlib inline

ModuleNotFoundError: No module named 'mysql'

In [ ]:
# Creating connection object
mydb = mysql.connector.connect(
    host = "P3NWPLSK12SQL-v12.shr.prod.phx3.secureserver.net",
    user = "datastudent",
    password = "dataprog2023!"
)